In [48]:
from gensim.parsing.preprocessing import remove_stopwords
from  gensim.parsing.porter import PorterStemmer
from gensim import corpora
from gensim import models
from gensim import similarities
import xml.etree.ElementTree as ET
import os
from smart_open import open

In [49]:
#funcion que retorna una lista con los docs extraidos de los NAF
def xml_extract(carpeta):
    dicc_documents = {}
    for archivo in os.listdir(carpeta):
        direc = carpeta + '/' + archivo
        doc = ET.parse(direc)
        root = doc.getroot()
        Ids = root.find('*/public').get("publicId")
        raw = root.find('raw').text
        dicc_documents[Ids] = raw
    return dicc_documents



In [50]:
#funcion que normaliza, quita stopwords y hace steming a los doc
def process(text):
    p = PorterStemmer()
    doc_org = text.lower() #normalizo en minusculas
    doc_nor = remove_stopwords(doc_org)
    doc_stem = p.stem_sentence(doc_nor)
    return doc_stem.split()

In [51]:
def load_docs():
    dic_docs = {}
    dic_query = {}
    carpeta = 'C:/Users/Felipe Diaz/Documents/GitHub/Programas_python/Procesamiento de lenguaje/Motor_busqueda/prueba'
    dic_docs = xml_extract(carpeta)
    carpeta = 'C:/Users/Felipe Diaz/Documents/GitHub/Programas_python/Procesamiento de lenguaje/Motor_busqueda/pquery'
    dic_query = xml_extract(carpeta)
    #se crea la lista con los documentos procesados
    docdict = []
    for doc in dic_docs:
        docdict.append(process(dic_docs[doc]))
    #print("\nveamos\n")
    #for indice in docdict:
    #    print(indice)
    #llamamos al metodo gensim para crear el diccionario 
    #a partir de los documentos
    vocabulary_doc = corpora.Dictionary(docdict)
    vocabulary_doc.save('dic_documents.dict')
    print(vocabulary_doc)
    print(vocabulary_doc.token2id)
    class mycorpus(object):
        def __iter__(self):
            for doc in dic_docs:
                yield vocabulary_doc.doc2bow(process(dic_docs[doc]))
    
    corpus_memory_friendly = mycorpus()

    corpora.MmCorpus.serialize('corpus.mm',corpus_memory_friendly)

    corpus = corpora.MmCorpus('corpus.mm')
    print("\ncorpus\n",corpus)

    #IDF Y TF
    vocabulary_doc = corpora.Dictionary.load('dic_documents.dict')
    corpus = corpora.MmCorpus('corpus.mm')
    tfidf = models.TfidfModel(corpus)
    





In [52]:
if __name__ == "__main__":
    load_docs()


Dictionary(656 unique tokens: ['1785,', '1812.', '1819.', '1820', '1822.']...)
{'1785,': 0, '1812.': 1, '1819.': 2, '1820': 3, '1822.': 4, '1833.': 5, '19-year-old': 6, '21,': 7, '6th,': 8, '83.': 9, '[1]': 10, '[2]': 11, 'accid': 12, 'acid': 13, 'activ': 14, 'affect': 15, 'ag': 16, 'alexi': 17, 'also,': 18, 'american': 19, 'armi': 20, 'basi': 21, 'beaumont': 22, 'beaumont:': 23, 'beaumont’': 24, 'best': 25, 'better': 26, 'bit': 27, 'book': 28, 'born': 29, 'born.': 30, 'break': 31, 'broken.': 32, 'caus': 33, 'chemic': 34, 'chemical,': 35, 'children': 36, 'close': 37, 'compani': 38, 'company.': 39, 'complet': 40, 'connecticut': 41, 'consid': 42, 'di': 43, 'did.': 44, 'differ': 45, 'digest': 46, 'digestion.': 47, 'digestion”': 48, 'discov': 49, 'earli': 50, 'emotions,': 51, 'examin': 52, 'exist': 53, 'expect': 54, 'experi': 55, 'famou': 56, 'find': 57, 'fistula.': 58, 'follow': 59, 'food': 60, 'fort': 61, 'french-canadian': 62, 'fur': 63, 'fuse': 64, 'gain': 65, 'gastric': 66, 'gave': 67